# Here's what Adam did....

#### Adam had a LOT of senate legislative data lying around. He tried to implement two machine learninng techniques: Classifers and linear regression.

#### The first section of this notebook is Adam importing and prepping some of the data before ML usage. Second section is trying to, hopefully somewhat succesfully, use a classifier to predict if a bill has been signed or not based on it's text.

#### The third section is a botched attempt at using linear regression to predict how long it would take a bill to get to the governor's desk based on it's language.

#### Import and cleaning. Some column addition and df arranging.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("2013SenateScraper2.0.csv",sep="|")

In [3]:
df.columns

Index(['bill_number', 'url', 'bill_title', 'spon_sen_name', 'dist_name',
       'aye_vote', 'nay_vote', 'last_legis_action', 'dtg_date',
       'signed_by_gov', 'signed_date', 'vetoed_by_gov', 'vetoed_date',
       'sponser_memo', 'sponser_memo2', 'bill_text'],
      dtype='object')

In [4]:
df.signed_by_gov.value_counts()

False    6646
True     1111
Name: signed_by_gov, dtype: int64

In [5]:
df_signed = df[df.signed_by_gov == True]

In [6]:
df_signed.describe()

,bill_number,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,last_legis_action,dtg_date,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text
count,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111,1111,1105,1111
unique,1111,1111,1090,59,24,31,20,77,55,1,95,1,1,1111,1086,1111
top,senate Bill S4235,https://www.nysenate.gov/legislation/bills/201...,Extends the provisions of rent control laws,Martin J. Golden,"(R, C, IP) 0 Senate District",62,0,"Jun 21, 2012","Jul 22, 2011",True,"Jul 18, 2012",False,no date,BILL NUMBER:S2914\n\nTITLE OF BILL:\nAn act\nt...,EFFECTIVE DATE:\nThis act shall take effect im...,S4627 (ACTIVE) - Bill Text\n \ndownlo...
freq,1,1,4,49,523,309,756,107,164,1111,156,1111,1111,1,6,1


In [7]:
df_signed = df_signed[~df_signed.last_legis_action.str.contains('no floor vote')]
df_signed = df_signed[~df_signed.dtg_date.str.contains('not delivered to governor')]

In [8]:
df_signed['last_legis_action'] = pd.to_datetime(df_signed['last_legis_action'])
df_signed['dtg_date'] = pd.to_datetime(df_signed['dtg_date'])

In [9]:
df_signed['total_time'] = df_signed['dtg_date'] - df_signed['last_legis_action']

In [10]:
df_signed.total_time.append

<bound method Series.append of 58     166 days
128     47 days
154     83 days
156      3 days
184      9 days
         ...   
7754    29 days
7767    92 days
7769    67 days
7771     1 days
7779    46 days
Name: total_time, Length: 1044, dtype: timedelta64[ns]>

In [11]:
df_signed.columns

Index(['bill_number', 'url', 'bill_title', 'spon_sen_name', 'dist_name',
       'aye_vote', 'nay_vote', 'last_legis_action', 'dtg_date',
       'signed_by_gov', 'signed_date', 'vetoed_by_gov', 'vetoed_date',
       'sponser_memo', 'sponser_memo2', 'bill_text', 'total_time'],
      dtype='object')

# Count words with a vectorizer

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer()
df = df.dropna()
matrix = vectorizer.fit_transform(df.bill_text)
bill_text_vectorized_df = pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names())

bill_text_vectorized_df

,00,000,0000,00000,00008,0001,00018586,00018587,00018588,0002,...,zoos,zostera,zpass,zuba,zuccolotto,zv,zygote,zz,zzz,zzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7731,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7732,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7733,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7734,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df['is_signed'] = (df['signed_by_gov'] == True).astype(int)
df.head()

,bill_number,url,bill_title,spon_sen_name,dist_name,aye_vote,nay_vote,last_legis_action,dtg_date,signed_by_gov,signed_date,vetoed_by_gov,vetoed_date,sponser_memo,sponser_memo2,bill_text,is_signed
0,senate Bill S21,https://www.nysenate.gov/legislation/bills/201...,Relates to payment of independent contractors,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,no floor vote,not delivered to governor,False,no date,False,no date,BILL NUMBER:S21\n\nTITLE OF BILL:\nAn act\nto ...,-Provides that the Department of Labor must pr...,S21 (ACTIVE) - Bill Text\n \ndownload...,0
1,senate Bill S22,https://www.nysenate.gov/legislation/bills/201...,Provides that the income level for the require...,Daniel L. Squadron,"(D, WF) 0 Senate District",59,0,no floor vote,not delivered to governor,False,no date,False,no date,BILL NUMBER:S22\n\nTITLE OF BILL:\nAn act\nto ...,"income persons, particularly senior citizens. ...",S22 (ACTIVE) - Bill Text\n \ndownload...,0
2,senate Bill S23,https://www.nysenate.gov/legislation/bills/201...,"Relates to the sale of tonic water, bitters an...",Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,no floor vote,not delivered to governor,False,no date,False,no date,BILL NUMBER:S23\n\nTITLE OF BILL:\nAn act\nto ...,Section 3- Effective date.\n\nJUSTIFICATION:\n...,S23 (ACTIVE) - Bill Text\n \ndownload...,0
3,senate Bill S24,https://www.nysenate.gov/legislation/bills/201...,Requires certification notice to tenant when r...,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,no floor vote,not delivered to governor,False,no date,False,no date,BILL NUMBER:S24\n\nTITLE OF BILL:\nAn act\nto ...,prevent such disputes from being elevated to t...,S24 (ACTIVE) - Bill Text\n \ndownload...,0
4,senate Bill S25,https://www.nysenate.gov/legislation/bills/201...,Relates to making the provisions governing liq...,Daniel L. Squadron,"(D, WF) 0 Senate District",Not voted on,Not voted on,no floor vote,not delivered to governor,False,no date,False,no date,BILL NUMBER:S25\n\nTITLE OF BILL:\nAn act\nto ...,EXISTING LAW:\nABC Law § 64{6-a) sets forth ce...,S25 (ACTIVE) - Bill Text\n \ndownload...,0


In [15]:
df.shape

(7736, 17)

### Classifier stuff. Random Forest, to be exact

In [16]:
X = bill_text_vectorized_df

y = df.is_signed

In [17]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100)
clf.fit(X, y)

RandomForestClassifier()

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [19]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5)

In [20]:
from sklearn.metrics import confusion_matrix

y_true = y_test
y_pred = clf.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['signed', 'not signed'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names)

,Predicted signed,Predicted not signed
Is signed,1633,35
Is not signed,200,66


In [21]:
clf.score(X,y)

0.8835315408479835

In [22]:
clf.score(X_test,y_test)

0.8784901758014477

In [32]:
pip install eli5

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\aefar\AppData\Local\Programs\Python\Python38\python.exe -m pip install --upgrade pip' command.


In [33]:
import eli5

feature_names = list(bill_text_vectorized_df.columns)
eli5.show_weights(clf, feature_names=feature_names, top=50)

Weight,Feature
0.1984,amending
0.1578,january
0.1557,adding
0.0931,1210
0.0638,parcel
0.0395,instal
0.0312,hempstead
0.0307,expire
0.0303,part
0.0264,skelos
